In [1]:
import pandas as pd
import numpy as np
from census_api import census_df
from scrape_realtor import realtor_df

/Users/miguelrondon/Desktop/CODE/Data Projects/Real-Estate-By-Census/scrape_realtor.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  realtor_df = pd.read_csv(


In [2]:
state_abrevs=pd.read_json('state_abrev.json', dtype="str")


In [3]:
if len(realtor_df["month_date_yyyymm"].unique()) == 1: 
    realtor_df["State"] = realtor_df["State"].str.upper().str.replace(" ", "")
    realtor_df["County"] = realtor_df["County"].str.capitalize()
    realtor_df = pd.merge(realtor_df, state_abrevs, how='left',
                      left_on="State", right_on="Code")
    realtor_df["state"] = realtor_df["State_y"]
    realtor_df = realtor_df.drop(['State_x', 'State_y', 'Abbrev'], axis=1)
    realtor_df.columns = realtor_df.columns.str.lower()
    #as_of_month_year = realtor_df["month_date_yyyymm"][0]
else: 
    raise Exception("the realtor dataframe has more than one date in it, it should only have one date. Please review...") 

In [4]:
for column in realtor_df:
    if realtor_df[column].dtype == pd.StringDtype:
        realtor_df[column] = realtor_df[column].str.strip()

In [5]:
# import datetime
# datetime = datetime.datetime.strptime(
#     as_of_month_year, "%Y%m").strftime("%b_%Y")



In [5]:
census_df['County'] = census_df['County'].str.replace("County", "")
census_df.head()

,County,State,Total:,Total:Population of one race:,Total:Population of one race:White alone,Total:Population of one race:Black or African American alone,Total:Population of one race:American Indian and Alaska Native alone,Total:Population of one race:Asian alone,Total:Population of one race:Native Hawaiian and Other Pacific Islander alone,Total:Population of one race:Some Other Race alone,Total:Population of two or more races:,Total:Hispanic or Latino
0,Autauga,Alabama,58805,55648,42160,11445,217,881,35,910,3157,2117
1,Baldwin,Alabama,231767,216743,189399,18217,1582,2067,143,5335,15024,12686
2,Barbour,Alabama,25223,24523,11317,11933,116,117,1,1039,700,1510
3,Bibb,Alabama,22293,21534,16555,4413,60,32,9,465,759,740
4,Blount,Alabama,59134,55478,50663,845,337,178,24,3431,3656,5771


In [6]:
census_df.columns = census_df.columns.str.lower().str.strip().str.replace(" ",
                                                                      "_").str.replace(":", "_").str.strip("_")

for column in census_df:
    if census_df[column].dtype == pd.StringDtype:
        census_df[column] = census_df[column].str.strip()

census_df["county"] = census_df["county"].str.capitalize()


In [7]:
#edge cases

#louisiana parishes, 
census_df['county'] = np.where(census_df['state'] == 'Louisiana', census_df['county'].str.replace(" parish", ""), 
np.where(census_df['state'] == 'Alaska', census_df['county'].str.replace(" borough", "").str.replace(" census area", "").str.replace(" municipality", "").str.replace(" city and borough", ""), census_df['county']))

#other edge cases to be handled later,

In [9]:
from sqlalchemy import create_engine 
from sqlalchemy_utils import database_exists, create_database
from postgres_credentials import alchemycred


engine= create_engine(alchemycred)
if not database_exists(engine.url):
    create_database(engine.url)


In [10]:
#constantly refreshes data when run 
realtor_df.to_sql("county_prices", engine, if_exists='replace')

#Does not refresh data once created. Data remains the same over time. 
census_df.to_sql("census_demographics", engine, if_exists='replace') 

221